In [ ]:
!apt install fonts-nanum*

In [ ]:
!sudo apt-get install -y fonts-nanum


In [ ]:
!rm ~/.cache/matplotlib -rf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
from matplotlib import font_manager, rc
import matplotlib as mpl
plt.rc('font', family='NanumBarunGothic') 


In [ ]:
#폰트, 드라이브, 패스 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font',family=font)


drive.mount('/content/drive')
path = '/content/drive/MyDrive/Colab Notebooks/데청캠2022/names'

In [ ]:
#1880년 신생아 이름 출력
df_names1880 = pd.read_csv(path+'/yob1880.txt',
                           names=['name','gender','number'])
df_names1880

In [ ]:
#1880년 신생아 수 
df_names1880.groupby('gender')['number'].sum()

In [ ]:
#전체 이름 불러오기
years = range(1880,2022)
pieces = list()
columns = ['name','gender','number']

for year in years:
    print(year,end=': ')
    df_names = pd.read_csv(path+'/yob'+str(year)+'.txt',names=columns)

    df_names['year']=year
    pieces.append(df_names) #exit for 

df_names_all = pd.concat(pieces, ignore_index=True)

In [ ]:
df_names_all

In [ ]:
#연도별 신생아 수 
#pivot_table 은 매개변수 index를 index 정보, columns = Columnes, values=에 특정행동(aggfunc)을 dataframe으로 생성
total_births=df_names_all.pivot_table('number',index='year',columns='gender',aggfunc=sum)

In [ ]:
total_births.plot(title="연도별 신생아 수")
plt.show()

In [ ]:
#각 이름이 전체 신생아 수에서 차지하는 비율 prop 
#메소드 groupby에서 적용할 메소드 add_prop 선언
def add_prop(group):
    group['prop']=group['number']/group['number'].sum()
    return group
#특정 년도가 담겨진 객체를 groupby후 메소드 연산 진행, 기존 객체에 새로운 열 prop추가후 새로운 객체 df_name_prop생성
df_names_prop = df_names_all.groupby(['year','gender']).apply(add_prop)

df_names_prop

In [ ]:
#비율의 합 출력
#연도/성별로 나눈 비율의 합이 1인지 확인, 객체 datagrame이 너무 길어서 요약되면 string문자열 형태로 변환 후 출력
df_names_prop.groupby(['year','gender'])['prop'].sum()

In [ ]:
#각 이름이 차지하는 비율 중 상위 1000개 year, gender로 groupby후 결과 
#생성 값은 number을 기준으로 내림차순 정렬후 상위 1000개를 추출
pieces = list()

for year_gender, group in df_names_prop.groupby(['year','gender']):
    pieces.append(group.sort_values(by='number',ascending=False)[:1000])
    top1000 = pd.concat(pieces, ignore_index=True)

In [ ]:
top1000

In [ ]:
#연도별 손호하는 신생아 이름 상위 1000개 남 녀
top1000_boys = top1000[top1000['gender']=='M']
top1000_girls = top1000[top1000['gender']=='F']

top1000_boys.head()
top1000_girls.head()

In [ ]:
#연도별 선호하는 이름 변화 top100에서 number을 기준으로 year/columns의 합계 계산 
# 그중 4개를 가져다가 변화량 봄
total_number = top1000.pivot_table('number',index='year',columns='name',aggfunc=sum)

subset=total_number[['John','James','Mary','Emma']]

subset.plot(subplots=True,figsize=(12,10),
            title="연도별 출생순")
plt.show()

In [ ]:
#신생아 이름의 다양성 분석 prop을 기준으로 year/gender의 합계 계산
#prop은 비율이므로 합계가 1을 넘지 않음
table=top1000.pivot_table('prop',index='year',columns='gender',aggfunc=sum)

In [ ]:
table.plot(title="연도별 상위 1000개 이름의 비율",yticks=np.linspace(0,1.2,13),xticks=range(1880,2030,10))
plt.show()

In [ ]:
#2021년 상위 1000개의 이름의 통계 
#prop으로 누적한 후 내림차순 정렬그리고 중앙 50%을 출력, 
df = top1000_boys[top1000_boys['year']==2021]
df_prop_cumsum=df.sort_values(by='prop',ascending=False)['prop'].cumsum()

df

In [ ]:
#메소드 seatchsorted는 객체의 매개변수 v에 인접한 위치를 알려줌
df_prop_cumsum[:10]
df_prop_cumsum.values.searchsorted(0.5)

In [ ]:
#1921년 상위 1000개의 이름의 통계
df = top1000_boys[top1000_boys['year']==1921]
df_prop_cumsum= df.sort_values(by='prop',ascending=False)['prop'].cumsum()
df
df_prop_cumsum[:10]
df_prop_cumsum.values.searchsorted(0.5)

In [ ]:
#다시 신생아 이름의 다양성 분석 
#상위 1000개의 이름이 담긴 객체 top1000을 그룹후 특정 메소드 연산을 진행
def get_quantile_count(group,q=0.5):
    group= group.sort_values(by='prop',ascending=False)
    return group.prop.cumsum().values.searchsorted(q)+1

In [ ]:
diversity=top1000.groupby(['year','gender']).apply(get_quantile_count)
diversity=diversity.unstack('gender')

diversity.plot(title="연도별 상위 1000개의 이름 중 50%가 가진 이름의 수")
plt.show()

In [ ]:
#신생아 이름의 마지막 글자의 변화 
#마지막 글자만 가져와서 index 정보 last_letters와 columns 정보 gender/year을 기준으로 number 값을 더한 객체 생성
get_last_letter = lambda x: x[-1]
last_letter= df_names_all['name'].map(get_last_letter)

last_letter['name']='last_letter'

table = df_names_all.pivot_table('number',index=last_letter,columns=['gender','year'],aggfunc=sum)
table

In [ ]:
#신생아 이름의 마지막 글자의 비율 
#특정년도 자료만 추출하고 이를 비율로 변경한 객체 letterprop 생성
subtable=table.reindex(columns=[1900,1930,1960,1990,2020],level='year')

letter_prop = subtable/subtable.sum()
letter_prop

In [ ]:
#다시 신생아 이름의 마지막 글자의 변화
# 각글자의 비율 계산후 letter_prop으로 막대 그래프 생성, bar, rot로 글씨의 기울기 설정(안할시 기우려서 나옴) 
fig, axes = plt.subplots(2,1,figsize=(20,12))
fig.suptitle('성별 마지막 글자 빈도수',fontsize=16)

letter_prop['M'].plot.bar(ax=axes[0], title='Male', rot=0)
letter_prop['F'].plot.bar(ax=axes[1], title='Female', rot=0)
plt.show()

In [ ]:
#신생아 이름 중 흔한 마지막 글자 d/e/n/y
letter_prop = table/table.sum()
deny_t = letter_prop.loc[['d','e','n','y'],'M'].T
deny_t.plot(title="연도별 d/e/n/y로 끝나는 남자 이름의 변화")
plt.show()

In [ ]:
#남자 신생아 이름과 여자 신생아 이름이 바뀐 경우 
#lesl이라는 문자열 사용 (전체 이름중 lesl이 들어간 문자열만 추출, 이름을 소문자로 바뀌어 연도 성별 무관하게 합)
all_names = pd.Series(top1000['name'].unique())
lesley_like = all_names[all_names.str.lower().str.contains('lesl')]

filtered = top1000[top1000['name'].isin(lesley_like)]
filtered.groupby('name')['number'].sum()

In [ ]:
#추출한 이름을 토대로 index 정보 year & columns의 gender 를 기준으로 number을 더한객체 테이블 생성
table = filtered.pivot_table('number', index='year',columns='gender',aggfunc='sum')
table= table.div(table.sum(1),axis=0)

table['M'].plot()
table['F'].plot()
plt.title('연도별 lesl라는 단어가 들어간 이름 비율')
plt.legend()
plt.show()